## Three soldiers trading

In [2]:
import warnings
warnings.filterwarnings('ignore')

import time
from datetime import date, datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas_ta as ta
from ps_candlestick import get_type_candlestick
from ps_candlestick import has_reversal_pattern

In [3]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [4]:
data = dataset.copy()
data["current"] = data.index + pd.DateOffset(minutes=5)
data['1st_body'] = data['Close'] - data['Open']
data['2nd_body'] = data['1st_body'].shift(1)
data['3rd_body'] = data['1st_body'].shift(2)
data.dropna(inplace=True)

## Strategy

In [5]:
def trade_simulation(prepared_data):
    _trades = pd.DataFrame(columns=("EntryTime", "EntryPrice", "ExitTime", "ExitPrice", "Type", "Profit"))
    has_open_deal = False
    type_open_deal = ''
    # Stoploss at x0 pips
    sl_step = 2
    # Takeprofit at x0 pips(R/R = 1/3)
    tp_step = 6
    for i, row in prepared_data.iterrows():
        if 100*row.name.hour + row.name.minute < 1420:
            if has_open_deal:
                if type_open_deal == 'short':
                    # Stoploss
                    if row['High'] > entry_price + sl_step:
                        profit = -sl_step
                        exit_price = entry_price + sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['Low'] < entry_price - tp_step:
                            profit = tp_step
                            exit_price = entry_price - tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                            has_open_deal = False
                            type_open_deal = ''
                elif type_open_deal == 'long':
                    # Stoploss
                    if row['Low'] < entry_price - sl_step:
                        profit = -sl_step
                        exit_price = entry_price - sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['High'] > entry_price + tp_step:
                            profit = tp_step
                            exit_price = entry_price + tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                            has_open_deal = False
                            type_open_deal = ''
                        #
            if not has_open_deal:
                if row['signal'] == 'short':
                    # Open short deal
                    entry_price = (row['Close'] + row['Open'])/2
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'short'
                elif row['signal'] == 'long':
                    # Open short deal
                    entry_price = (row['Close'] + row['Open'])/2
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'long'
        else:
            if 100*row.name.hour + row.name.minute == 1425:
                if has_open_deal:
                    # close open deal at 2:25PM (dataframe)
                    if type_open_deal == 'short':
                        profit = entry_price - row['Close']
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    elif type_open_deal == 'long':
                        profit = row['Close'] - entry_price
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
    return _trades

In [7]:
def cal_signal_v1(row):
    signal = ''
    if row['1st_body'] > 0 and row['2nd_body'] > 0 and row['3rd_body'] > 0:
        signal = 'long'
    elif row['1st_body'] < 0 and row['2nd_body'] < 0 and row['3rd_body'] < 0:
        signal = 'short'
    return signal
data['signal'] = data.apply(lambda r: cal_signal_v1(r), axis=1)

In [8]:
trade_result = trade_simulation(data)

In [11]:
total_trades = len(trade_result)
profit = trade_result.Profit.sum()
trading_fee = total_trades * 0.3
win_rate = len(trade_result[trade_result.Profit > 0]) / total_trades
display_data = {
    'trades' : total_trades,
    'profit' : profit,
    'win rate' : win_rate,
    'trading_fee' : trading_fee,
    'profit_after_fee' : profit - trading_fee
}
display_data

{'trades': 5226,
 'profit': 5918.449999999999,
 'win rate': 0.42479908151549944,
 'trading_fee': 1567.8,
 'profit_after_fee': 4350.649999999999}

In [10]:
trade_result

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
0,2018-08-13 09:50:00,945.50,2018-08-13 10:45:00,943.50,Long,-2.00
1,2018-08-13 11:05:00,944.60,2018-08-13 14:05:00,950.60,Long,6.00
2,2018-08-14 09:50:00,956.05,2018-08-14 13:40:00,958.05,Short,-2.00
3,2018-08-14 13:40:00,957.80,2018-08-14 14:25:00,960.10,Long,2.30
4,2018-08-15 09:10:00,959.85,2018-08-15 10:50:00,957.85,Long,-2.00
...,...,...,...,...,...,...
5221,2024-07-10 09:30:00,1316.20,2024-07-10 09:40:00,1318.20,Short,-2.00
5222,2024-07-10 10:05:00,1319.50,2024-07-10 10:50:00,1317.50,Long,-2.00
5223,2024-07-10 10:55:00,1317.55,2024-07-10 11:05:00,1319.55,Short,-2.00
5224,2024-07-10 11:25:00,1319.35,2024-07-10 13:05:00,1317.35,Long,-2.00
